In [2]:
file_path="E:\Dataset\met_waves\Swan_cropped\swanSula201701_cropped.nc"

In [3]:
import netCDF4 as nc

def read_nc_info(file_path):
    # 打开 nc 文件
    dataset = nc.Dataset(file_path, 'r')

    # 获取所有全局属性
    global_attrs = {attr: dataset.getncattr(attr) for attr in dataset.ncattrs()}
    
    # 获取所有变量
    variables = list(dataset.variables.keys())

    # 获取经纬度
    lat = dataset.variables.get('latitude') or dataset.variables.get('lat')
    lon = dataset.variables.get('longitude') or dataset.variables.get('lon')
    time = dataset.variables.get('time')

    # 读取时间数据
    time_values = time[:].data if time is not None else None
    time_step = None
    if time_values is not None and len(time_values) > 1:
        time_step = time_values[1] - time_values[0]  # 计算时间间隔

    # 获取每个变量的单位、形状
    variable_info = {}
    for var_name in variables:
        var = dataset.variables[var_name]
        variable_info[var_name] = {
            "shape": var.shape,
            "units": var.units if "units" in var.ncattrs() else "N/A",
            "long_name": var.long_name if "long_name" in var.ncattrs() else var_name
        }

    # 打印信息
    print("🌍 文件全局属性:")
    for key, value in global_attrs.items():
        print(f"  {key}: {value}")

    print("\n📍 经纬度信息:")
    print(f"  纬度范围: {lat[:].min()} ~ {lat[:].max()}" if lat is not None else "  未找到纬度数据")
    print(f"  经度范围: {lon[:].min()} ~ {lon[:].max()}" if lon is not None else "  未找到经度数据")

    print("\n⏳ 时间信息:")
    print(f"  共有 {len(time_values)} 个时间步")
    print(f"  采样时间步长: {time_step}" if time_step is not None else "  未找到时间数据或无法计算步长")

    print("\n🌊 变量信息:")
    for var_name, info in variable_info.items():
        print(f"  {var_name}: {info['long_name']} (单位: {info['units']}, 形状: {info['shape']})")

    # 关闭文件
    dataset.close()

read_nc_info(file_path)


🌍 文件全局属性:
  Conventions: CF-1.5
  History: Thu Sep 30 11:31:36 2021: ncks -4 /lustre/storeB/project/fou/om/SWAN/Sula/INNER/Ut/swani_20170101.nc /lustre/storeB/project/SVV/E39/model/SWAN250/Sula/swanSula201701.nc
Created with agioncmd version 1.4
  Directional_convention: nautical
  project: Sula i
  run: T24
  NCO: 4.7.2
  institution: Norwegian Meteorological Institute, MET Norway
  title: Spectral wave simulations in Norwegian fjords
  summary: Hindcast simulations with the wave model SWAN (https://www.tudelft.nl/en/ceg/about-faculty/departments/hydraulic-engineering/sections/environmental-fluid-mechanics/research/swan) nested into the wave model WAM (hindcast data set NORA10, Reistad et al., 2011). Winds from WRF at 500m grid resolution are provided by Kjeller Vindteknikk/Norconsult. The grid spacing is 250m.
  acknowledgement: The simulations are performed under collaboration with the Norwegian Public Roads Administration connected to the Coastal Highway E39-project in mid-Norway.


In [4]:
import netCDF4 as nc

def list_nc_variables(file_path):
    # 打开 nc 文件
    dataset = nc.Dataset(file_path, 'r')

    # 获取所有变量
    variables = list(dataset.variables.keys())

    # 输出变量信息
    print("📌 文件中的所有变量：")
    for var_name in variables:
        var = dataset.variables[var_name]
        print(f"  - {var_name} (形状: {var.shape}, 维度: {var.dimensions})")

    dataset.close()

# 示例使用
list_nc_variables(file_path)


📌 文件中的所有变量：
  - hs (形状: (744, 128, 128), 维度: ('time', 'latitude', 'longitude'))
  - tm02 (形状: (744, 128, 128), 维度: ('time', 'latitude', 'longitude'))
  - theta0 (形状: (744, 128, 128), 维度: ('time', 'latitude', 'longitude'))
  - time (形状: (744,), 维度: ('time',))
  - longitude (形状: (128,), 维度: ('longitude',))
  - latitude (形状: (128,), 维度: ('latitude',))


In [3]:
import numpy as np
# 提取数据
import xarray as xr
ds = xr.open_dataset(file_path)
hs = ds['hs'].values  # (744, 128, 128)
tm02 = ds['tm02'].values  # (744, 128, 128)
theta0 = ds['theta0'].values  # (744, 128, 128)

# 合并为 3 通道
data = np.stack([hs, tm02, theta0], axis=-1)  # 形状 (744, 128, 128, 3)

print("数据形状:", data.shape)  # 输出 (T, W, H, C)

数据形状: (744, 128, 128, 3)


In [4]:
import xarray as xr
import numpy as np
import os

# 读取数据
dataset = xr.open_dataset(file_path)

# 提取变量
time = dataset['time'].values  # (744,)
hs = dataset['hs'].values  # (744, 128, 128)
tm02 = dataset['tm02'].values  # (744, 128, 128)
theta0 = dataset['theta0'].values  # (744, 128, 128)

# 划分训练集、验证集和测试集
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

num_samples = hs.shape[0]
train_end = int(num_samples * train_ratio)
val_end = int(num_samples * (train_ratio + val_ratio))

train_data = {
    'hs': hs[:train_end],
    'tm02': tm02[:train_end],
    'theta0': theta0[:train_end]
}
val_data = {
    'hs': hs[train_end:val_end],
    'tm02': tm02[train_end:val_end],
    'theta0': theta0[train_end:val_end]
}
test_data = {
    'hs': hs[val_end:],
    'tm02': tm02[val_end:],
    'theta0': theta0[val_end:]
}

# 保存数据
output_dir = "E:/Dataset/met_waves/Swan4predRNN"
os.makedirs(output_dir, exist_ok=True)

np.save(os.path.join(output_dir, "train.npy"), train_data)
np.save(os.path.join(output_dir, "val.npy"), val_data)
np.save(os.path.join(output_dir, "test.npy"), test_data)

dataset.close()
print("数据已成功划分并保存！")


数据已成功划分并保存！


In [2]:
import numpy as np

def check_nan_in_data(data):
    if np.isnan(data).any():
        print("⚠️ 数据中存在 NaN！")
        exit()
# 假设 `input_data` 是你的输入数据
input_data = np.load("E:/Dataset/met_waves/Swan4predRNN/train.npy",allow_pickle=True).item()
check_nan_in_data(input_data)


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''